In [1]:
import pandas as pd

team_data = pd.read_csv('../data/interim/NBA_team_stats_1979-2024_clean.csv')
player_data = pd.read_csv('../data/interim/kaggle_data_cleaned.csv')

In [2]:
team_data

,Unnamed: 0,Season,Team,W,G,L,W%,FG%,2P%,3P%,...,FTA/G,ORB/G,DRB/G,TRB/G,AST/G,STL/G,BLK/G,TOV/G,PF/G,PTS/G
0,0,2015,GSW,73,82,9,0.890,0.487,0.528,0.416,...,21.829268,9.951220,36.243902,46.195122,28.939024,8.402439,6.073171,15.182927,20.743902,114.890244
1,1,1995,CHI,72,82,10,0.878,0.478,0.496,0.403,...,24.439024,15.207317,29.402439,44.609756,24.792683,9.085366,4.207317,14.329268,22.036585,105.182927
2,2,1996,CHI,69,82,13,0.841,0.473,0.499,0.373,...,22.536585,15.060976,30.012195,45.073171,26.121951,8.731707,4.048780,13.524390,19.719512,103.146341
3,3,2016,GSW,67,82,15,0.817,0.495,0.557,0.383,...,22.560976,9.390244,35.036585,44.426829,30.378049,9.573171,6.768293,14.768293,19.329268,115.890244
4,4,2015,SAS,67,82,15,0.817,0.484,0.515,0.375,...,20.390244,9.390244,34.524390,43.914634,24.512195,8.256098,5.914634,13.060976,17.475610,103.536585
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1195,1200,1997,VAN,19,82,63,0.232,0.458,0.473,0.362,...,26.158537,13.182927,28.207317,41.390244,23.902439,7.487805,4.256098,17.134146,22.060976,96.621951
1196,1201,1992,MIN,19,82,63,0.232,0.466,0.483,0.292,...,27.402439,11.463415,26.878049,38.341463,24.402439,7.914634,5.548780,17.341463,24.731707,98.121951
1197,1202,1989,CHH,19,82,63,0.232,0.455,0.466,0.336,...,23.987805,11.731707,26.963415,38.695122,25.365854,9.487805,3.195122,14.951220,23.036585,100.390244
1198,1203,1987,NJN,19,82,63,0.232,0.468,0.480,0.301,...,28.146341,13.109756,27.585366,40.695122,21.890244,8.865854,4.695122,18.329268,24.902439,100.426829


In [3]:
player_data['rank'] = player_data.groupby(['season', 'tm'])['mp'].rank(ascending=False)
top_players = player_data[player_data['rank'] <= 7]

In [4]:
top_player_aggregates = top_players.groupby(['season', 'tm']).agg(
    avg_age = ('age', 'mean'),
    avg_experience = ('experience', 'mean'),
    total_points = ('pts_per_100_poss', 'sum'),
    total_assists = ('ast_per_100_poss', 'sum'),
    total_rebounds = ('trb_per_100_poss', 'sum'),
    total_steals = ('stl_per_100_poss', 'sum'),
    total_blocks = ('blk_per_100_poss', 'sum'),
    total_turnovers = ('tov_per_100_poss', 'sum'), 
    avg_o_rtg = ('o_rtg', 'mean'), 
    avg_d_rtg = ('d_rtg', 'mean')
).reset_index()

merged_data_top_players = pd.merge(team_data, top_player_aggregates, left_on=['Season', 'Team'], right_on=['season', 'tm'])
merged_data = merged_data_top_players.drop(columns=['season', 'tm', 'Unnamed: 0'])

merged_data.head()

,Season,Team,W,G,L,W%,FG%,2P%,3P%,FT%,...,avg_age,avg_experience,total_points,total_assists,total_rebounds,total_steals,total_blocks,total_turnovers,avg_o_rtg,avg_d_rtg
0,2015,GSW,73,82,9,0.890,0.487,0.528,0.416,0.763,...,26.000000,6.142857,164.5,41.2,69.6,12.0,9.4,21.8,112.142857,101.285714
1,2016,GSW,67,82,15,0.817,0.495,0.557,0.383,0.788,...,27.285714,7.142857,154.4,41.0,64.2,10.9,9.1,19.8,116.142857,104.000000
2,2015,SAS,67,82,15,0.817,0.484,0.515,0.375,0.803,...,29.000000,8.714286,156.0,36.0,69.1,11.7,8.9,20.0,111.285714,101.714286
3,2014,GSW,67,82,15,0.817,0.478,0.514,0.398,0.768,...,25.285714,5.142857,156.7,29.6,76.2,10.6,9.0,21.3,107.857143,102.142857
4,2006,DAL,67,82,15,0.817,0.467,0.491,0.381,0.805,...,27.714286,6.857143,164.4,31.8,65.6,11.5,7.2,20.6,113.285714,105.428571


In [5]:
merged_data.columns

Index(['Season', 'Team', 'W', 'G', 'L', 'W%', 'FG%', '2P%', '3P%', 'FT%',
       'TS%', 'eFG%', 'MP/G', 'FG/G', 'FGA/G', '2P/G', '2PA/G', '3P/G',
       '3PA/G', 'FT/G', 'FTA/G', 'ORB/G', 'DRB/G', 'TRB/G', 'AST/G', 'STL/G',
       'BLK/G', 'TOV/G', 'PF/G', 'PTS/G', 'avg_age', 'avg_experience',
       'total_points', 'total_assists', 'total_rebounds', 'total_steals',
       'total_blocks', 'total_turnovers', 'avg_o_rtg', 'avg_d_rtg'],
      dtype='object')

In [6]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler

X = merged_data.drop(columns=['Season', 'Team', 'W', 'G', 'L', 'W%', 'FG%', '2P%', '3P%', 'FT%',
       'TS%', 'MP/G', 'FG/G', 'FGA/G', '2P/G', '2PA/G', '3P/G',
       '3PA/G', 'FT/G', 'FTA/G', 'ORB/G', 'DRB/G', 'TRB/G', 'AST/G', 'STL/G',
       'BLK/G', 'TOV/G', 'PF/G'])
y = merged_data['W%']

scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)
X_scaled = pd.DataFrame(X_scaled, columns=X.columns)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=15)

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

mae, mse, r2

(0.08301739463596917, 0.01043091060829349, 0.458399107381944)

In [7]:
import numpy as np

feature_importance = model.coef_
features = X.columns

feature_importance_df = pd.DataFrame({
    'Feature': features,
    'Importance': feature_importance
}).sort_values(by='Importance', ascending=False)

feature_importance_df

,Feature,Importance
0,eFG%,0.090373
10,avg_o_rtg,0.035223
2,avg_age,0.020558
9,total_turnovers,0.011139
8,total_blocks,0.008363
5,total_assists,0.005631
7,total_steals,0.003859
4,total_points,-0.004369
6,total_rebounds,-0.013804
1,PTS/G,-0.015379
